In [1]:
import pandas as pd
df = pd.read_csv('ir_data.csv')

We begin to work on the first 1000 data sets in the beginning just to get the structure working

In [124]:
x = df.as_matrix()
x = x[:1001,:]
print(x)

[[0 174 22532 ..., 'RT LOWER LOBE PNEUMONIA' 0 1]
 [1 170 22532 ..., 'RT LOWER LOBE PNEUMONIA' 0 1]
 [2 59795 22532 ..., 'RT LOWER LOBE PNEUMONIA' 0 1]
 ..., 
 [998 59810 26175 ..., 'RESPIRATORY DISTRESS' 0 1]
 [999 59811 26175 ..., 'RESPIRATORY DISTRESS' 0 1]
 [1000 105662 26175 ..., 'RESPIRATORY DISTRESS' 0 1]]


In [49]:
list(df)
for num, col_name in enumerate(df):
    print(num, col_name)

0 Unnamed: 0
1 ROW_ID_x
2 SUBJECT_ID
3 HADM_ID
4 CHARTDATE
5 CHARTTIME
6 STORETIME
7 CATEGORY
8 DESCRIPTION
9 CGID
10 ISERROR
11 TEXT
12 ROW_ID_y
13 ADMITTIME
14 DISCHTIME
15 DEATHTIME
16 ADMISSION_TYPE
17 ADMISSION_LOCATION
18 DISCHARGE_LOCATION
19 INSURANCE
20 LANGUAGE
21 RELIGION
22 MARITAL_STATUS
23 ETHNICITY
24 EDREGTIME
25 EDOUTTIME
26 DIAGNOSIS
27 HOSPITAL_EXPIRE_FLAG
28 HAS_CHARTEVENTS_DATA


### SUBJECT_ID, HADM_ID
Identifiers which specify the patient: SUBJECT_ID is unique to a patient and HADM_ID is unique to a patient hospital stay.

### CG_ID 
This refers to the ID of the caregivere who made the observation and this is pretty irrelevant to us.

### STORE_TIME
This refers to the time at which the observation was actually recorded in the EHR. We shall use this time as CHART_TIME is not actually avalaible in the EHR.

### CATEGORY, DESCRIPTION
CATEGORY and DESCRIPTION define the type of note recorded. For example, a CATEGORY of ‘Discharge’ indicates that the note is a discharge note, and a DESCRIPTION of ‘Summary’ in conjunction with this indicates that the note is a discharge summary

### EDREGTIME, EDOUTTIME

Time that the patient was registered and discharged from the emergency department

### HOSPITAL_EXPIRE_FLG
Whether or not the patient deid in the hospital

### The columns that we consider are important for the initial part of the project
* 2 SUBJECT_ID
* 3 HADM_ID
* 6 STORETIME
* 7 CATEGORY
* 8 DESCRIPTION
* 11 TEXT
* 13 ADMITTIME
* 14 DISCHTIME
* 15 DEATHTIME
* 16 ADMISSION_TYPE
* 26 DIAGNOSIS
* 27 HOSPITAL_EXPIRE_FLAG

In [50]:
pat_dict = {}
# for rows in x:
#     pat_id = rows[2]
#     pat_dict[pat_id] = pat_dict.get(pat_id, 0)+1
#     print(pat_id, pat_dict[pat_id])
count = 0
count2 = 0
for items in (df['ISERROR']):
    if pd.isnull(items) == False:
        count2 += 1
    else:
        count += 1


We check the number of data entries that have chart events and the entries that don't.

In [51]:
count1 = 0
count2 = 0
for items in (df['HAS_CHARTEVENTS_DATA']):
    if items==1:
        count1 += 1
    else:
        count2 += 1
print(count1, count2)

1842222 9122


In [52]:
count = 1
patid_set = set()
patid = {}
for items in x:
    patid_set.add(items[2])
    patid[items[2]] = patid.get(items[2], 0)+1

In [192]:
# Code to check if multiple diagnosis occur for the same patient, if so how many times?
pat_diag_dict = {}
for items in x:
    pat_id = items[2]
    diag = items[26]
    if pat_id not in pat_diag_dict:
        pat_diag_dict[pat_id] = diag
    else:
        print(pat_diag_dict[pat_id], diag, items[3])
        
# Hence we conclude that there are cases when we have had different diagnosis for the same pati_id, hadm_id

RT LOWER LOBE PNEUMONIA RT LOWER LOBE PNEUMONIA 167853.0
RT LOWER LOBE PNEUMONIA RT LOWER LOBE PNEUMONIA 167853.0
RT LOWER LOBE PNEUMONIA RT LOWER LOBE PNEUMONIA 167853.0
RT LOWER LOBE PNEUMONIA RT LOWER LOBE PNEUMONIA 167853.0
RT LOWER LOBE PNEUMONIA RT LOWER LOBE PNEUMONIA 167853.0
RT LOWER LOBE PNEUMONIA RT LOWER LOBE PNEUMONIA 167853.0
RT LOWER LOBE PNEUMONIA RT LOWER LOBE PNEUMONIA 167853.0
RT LOWER LOBE PNEUMONIA RT LOWER LOBE PNEUMONIA 167853.0
RT LOWER LOBE PNEUMONIA RT LOWER LOBE PNEUMONIA 167853.0
RT LOWER LOBE PNEUMONIA RT LOWER LOBE PNEUMONIA 167853.0
RT LOWER LOBE PNEUMONIA RT LOWER LOBE PNEUMONIA 167853.0
RT LOWER LOBE PNEUMONIA RT LOWER LOBE PNEUMONIA 167853.0
RT LOWER LOBE PNEUMONIA RT LOWER LOBE PNEUMONIA 167853.0
RT LOWER LOBE PNEUMONIA RT LOWER LOBE PNEUMONIA 167853.0
RT LOWER LOBE PNEUMONIA RT LOWER LOBE PNEUMONIA 167853.0
RT LOWER LOBE PNEUMONIA RT LOWER LOBE PNEUMONIA 167853.0
RT LOWER LOBE PNEUMONIA RT LOWER LOBE PNEUMONIA 167853.0
RT LOWER LOBE PNEUMONIA RT LOWE

CONGESTIVE HEART FAILURE CONGESTIVE HEART FAILURE 121936.0
CONGESTIVE HEART FAILURE CONGESTIVE HEART FAILURE 121936.0
CONGESTIVE HEART FAILURE CONGESTIVE HEART FAILURE 121936.0
COLITIS COLITIS 139574.0
COLITIS COLITIS 139574.0
COLITIS COLITIS 139574.0
COLITIS COLITIS 139574.0
COLITIS COLITIS 139574.0
COLITIS COLITIS 139574.0
COLITIS COLITIS 139574.0
COLITIS COLITIS 139574.0
COLITIS COLITIS 139574.0
COLITIS COLITIS 139574.0
COLITIS COLITIS 139574.0
COLITIS COLITIS 139574.0
COLITIS COLITIS 139574.0
COLITIS COLITIS 139574.0
COLITIS COLITIS 139574.0
COLITIS COLITIS 139574.0
COLITIS COLITIS 139574.0
COLITIS COLITIS 139574.0
COLITIS COLITIS 139574.0
COLITIS COLITIS 139574.0
COLITIS COLITIS 139574.0
CORONARY ARTERY DISEASE CORONARY ARTERY DISEASE 169684.0
CORONARY ARTERY DISEASE CORONARY ARTERY DISEASE 169684.0
CORONARY ARTERY DISEASE CORONARY ARTERY DISEASE 169684.0
CORONARY ARTERY DISEASE CORONARY ARTERY DISEASE 169684.0
CORONARY ARTERY DISEASE CORONARY ARTERY DISEASE 169684.0
CORONARY ARTE

In [184]:
# data = [{"pat_id":None}]
# data[0]['pat_id'] = 2
# data.append({'pat_id':3})
# l = 5
# desc = [items['pat_id'] for items in data]
# if l not in desc:
#     data.append({'pat_id':l})
# # print(data)
pat_set = set()
final_structure = []
for entry in x:
    pat_id = entry[2]
    stay_id = entry[3]
    diagnosis = entry[26]
    store_time = entry[6]
    if pat_id not in pat_set:
        # then we need to construct a new tuple with the entries
        pat_set.add(pat_id)
        new_struct = {'pat_id':pat_id, 'hospital':[]}

        stay_struct = {'stay_id':stay_id, 'diagnosis':[diagnosis], 'time':[store_time]} 
        # we make diagnosis a list cause we might have differnt diagnosis for the same stay
        new_struct['hospital'].append(stay_struct)
        final_structure.append(new_struct)
final_arr = [data['pat_id'] for data in final_structure]
for items in final_structure:
    print(items)

{'pat_id': 22532, 'hospital': [{'stay_id': 167853.0, 'diagnosis': ['RT LOWER LOBE PNEUMONIA'], 'time': [nan]}]}
{'pat_id': 13702, 'hospital': [{'stay_id': 107527.0, 'diagnosis': ['CHRONIC OBSTRUCTIVE PULMONARY DISEASE'], 'time': [nan]}]}
{'pat_id': 26880, 'hospital': [{'stay_id': 135453.0, 'diagnosis': ['S/P FALL;TELEMETRY'], 'time': [nan]}]}
{'pat_id': 53181, 'hospital': [{'stay_id': 170490.0, 'diagnosis': ['RIGHT TEMPORAL MENIGIOMA/SDA'], 'time': [nan]}]}
{'pat_id': 20646, 'hospital': [{'stay_id': 134727.0, 'diagnosis': ['PNEUMONIA;HYPOXIA'], 'time': [nan]}]}
{'pat_id': 42130, 'hospital': [{'stay_id': 114236.0, 'diagnosis': ['LEFT SPHENOID MENENGIOMA/SDA'], 'time': [nan]}]}
{'pat_id': 56174, 'hospital': [{'stay_id': 163469.0, 'diagnosis': ['BRAIN MASS/SDA'], 'time': [nan]}]}
{'pat_id': 28063, 'hospital': [{'stay_id': 121936.0, 'diagnosis': ['CONGESTIVE HEART FAILURE'], 'time': [nan]}]}
{'pat_id': 1136, 'hospital': [{'stay_id': 139574.0, 'diagnosis': ['COLITIS'], 'time': [nan]}]}
{'pa

print(x)

In [181]:
print (x[2:3, 11:12])

[[ 'PATIENT/TEST INFORMATION:\nIndication: Aortic valve disease. ?endocarditis\nHeight: (in) 67\nWeight (lb): 110\nBSA (m2): 1.57 m2\nBP (mm Hg): 106/72\nStatus: Inpatient\nDate/Time: [**2151-8-2**] at 17:09\nTest: Portable TTE(Complete)\nDoppler: Complete pulse and color flow\nContrast: None\nTechnical Quality: Adequate\n\n\nINTERPRETATION:\n\nFindings:\n\nLEFT ATRIUM: The left atrium is moderately dilated. The left atrium is\nelongated.\n\nRIGHT ATRIUM/INTERATRIAL SEPTUM: The right atrium is markedly dilated.\n\nLEFT VENTRICLE: Left ventricular wall thicknesses are normal. The left\nventricular cavity size is normal. There is mild global left ventricular\nhypokinesis. Overall left ventricular systolic function is moderately\ndepressed.\n\nRIGHT VENTRICLE: The right ventricular free wall is hypertrophied. The right\nventricular cavity is moderately dilated. There is moderate global right\nventricular free wall hypokinesis.\n\nAORTA: The aortic root is normal in diameter.\n\nAORTIC VAL